In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/kaggle-baidu/baidu_95_handle.csv


In [2]:
!pip install tensorflow-gpu==1.15.0

     |████████████████████████████████| 411.5MB 27kB/s 
     |████████████████████████████████| 512kB 23.5MB/s 
     |████████████████████████████████| 3.8MB 30.0MB/s 
ERROR: tensorflow 2.1.0rc0 has requirement tensorboard<2.1.0,>=2.0.0, but you'll have tensorboard 1.15.0 which is incompatible.
ERROR: tensorflow 2.1.0rc0 has requirement tensorflow-estimator<2.1.0,>=2.0.0, but you'll have tensorflow-estimator 1.15.1 which is incompatible.
  Found existing installation: tensorflow-estimator 2.0.1
    Uninstalling tensorflow-estimator-2.0.1:
      Successfully uninstalled tensorflow-estimator-2.0.1
  Found existing installation: tensorboard 2.0.2
    Uninstalling tensorboard-2.0.2:
      Successfully uninstalled tensorboard-2.0.2


In [3]:
!wget -nc https://storage.googleapis.com/bert_models/2018_11_03/chinese_L-12_H-768_A-12.zip
!unzip -o chinese_L-12_H-768_A-12.zip && rm chinese_L-12_H-768_A-12.zip

--2020-01-03 05:59:45--  https://storage.googleapis.com/bert_models/2018_11_03/chinese_L-12_H-768_A-12.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.203.128, 2607:f8b0:400c:c07::80
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.203.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 381892918 (364M) [application/zip]
Saving to: ‘chinese_L-12_H-768_A-12.zip’

chinese_L-12_H-768_ 100%[===================>] 364.20M  80.1MB/s    in 4.1s    

2020-01-03 05:59:49 (87.8 MB/s) - ‘chinese_L-12_H-768_A-12.zip’ saved [381892918/381892918]

Archive:  chinese_L-12_H-768_A-12.zip
   creating: chinese_L-12_H-768_A-12/
  inflating: chinese_L-12_H-768_A-12/bert_model.ckpt.meta  
  inflating: chinese_L-12_H-768_A-12/bert_model.ckpt.data-00000-of-00001  
  inflating: chinese_L-12_H-768_A-12/vocab.txt  
  inflating: chinese_L-12_H-768_A-12/bert_model.ckpt.index  
  inflating: chinese_L-12_H-768_A-12/bert_config.json  


In [4]:
##install bert if not already done
!pip install bert-tensorflow

     |████████████████████████████████| 71kB 2.1MB/s 


In [5]:
import os
import collections
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
from datetime import datetime

import bert
from bert import run_classifier
from bert import optimization
from bert import tokenization
from bert import modeling

In [6]:
##use downloaded model, change path accordingly
BERT_VOCAB= './chinese_L-12_H-768_A-12/vocab.txt'
BERT_INIT_CHKPNT = './chinese_L-12_H-768_A-12/bert_model.ckpt'
BERT_CONFIG = './chinese_L-12_H-768_A-12/bert_config.json'

In [7]:
tokenization.validate_case_matches_checkpoint(True,BERT_INIT_CHKPNT)
tokenizer = tokenization.FullTokenizer(
      vocab_file=BERT_VOCAB, do_lower_case=True)

tokenizer.tokenize("下列有关生物体内信息传递的叙述")

['下', '列', '有', '关', '生', '物', '体', '内', '信', '息', '传', '递', '的', '叙', '述']

In [8]:
##change path accordingly
data_path='/kaggle/input/kaggle-baidu/baidu_95_handle.csv'
data = pd.read_csv(data_path)
# 打乱
data.sample(frac=1).reset_index(drop=True)
data.rename(columns={'id':'label', 'labels':'content', 'content':'id'}, inplace = True)
data = data.drop(['label'], axis=1)
order= ['id','content','高中', '生物', '历史', '地理', '政治', '生物技术实践', '公民道德与伦理常识', '经济学常识',
              '生活中的法律常识', '现代生物技术专题', '科学社会主义常识',
              '地球与地图', '近代史', '遗传与进化', '生物科学与社会', '分子与细胞', '人口与城市',
              '宇宙中的地球', '生产活动与地域联系', '古代史', '现代史', '稳态与环境',
              '生物性污染', '细胞有丝分裂不同时期的特点', '液泡的结构和功能', '组成细胞的化学元素',
              '兴奋在神经纤维上的传导', '不完全显性', '免疫系统的组成', '生物技术在其他方面的应用', '群落的结构',
              '中央官制——三公九卿制', '核糖体的结构和功能', '人体免疫系统在维持稳态中的作用', '皇帝制度',
              '激素调节', '伴性遗传',
              '地球运动的地理意义', '地球运动的基本形式', '基因工程的原理及技术', '体液免疫的概念和过程',
              '基因的分离规律的实质及应用', '蛋白质的合成', '地球的内部圈层结构及特点', '人口增长与人口问题',
              '劳动就业与守法经营', '器官移植', '垄断组织的出现', '基因工程的概念', '神经调节和体液调节的比较',
              '组成细胞的化合物', '文艺的春天', '生物工程技术', '基因的自由组合规律的实质及应用', '郡县制', '人体水盐平衡调节',
              '内质网的结构和功能', '人体的体温调节', '免疫系统的功能', '与细胞分裂有关的细胞器', '太阳对地球的影响',
              '清末民主革命风潮', '复等位基因', '人工授精、试管婴儿等生殖技术', '“重农抑商”政策', '生态系统的营养结构',
              '减数分裂的概念', '地球的外部圈层结构及特点', '细胞的多样性和统一性', '工业区位因素', '细胞大小与物质运输的关系',
              '夏商两代的政治制度', '农业区位因素', '溶酶体的结构和功能', '内环境的稳态', '胚胎移植',
              '第三产业的兴起和“新经济”的出现', '中心体的结构和功能', '社会主义市场经济的伦理要求', '选官、用官制度的变化',
              '减数分裂与有丝分裂的比较', '遗传的细胞基础', '地球所处的宇宙环境', '培养基与无菌技术', '高尔基体的结构和功能',
              '社会主义是中国人民的历史性选择', '人口迁移与人口流动', '走进细胞', '避孕的原理和方法', '血糖平衡的调节',
              '海峡两岸关系的发展', '生命活动离不开细胞', '兴奋在神经元之间的传递', '拉马克的进化学说', '遗传的分子基础']
data = data[order]
data.head()

,id,content,高中,生物,历史,地理,政治,生物技术实践,公民道德与伦理常识,经济学常识,...,社会主义是中国人民的历史性选择,人口迁移与人口流动,走进细胞,避孕的原理和方法,血糖平衡的调节,海峡两岸关系的发展,生命活动离不开细胞,兴奋在神经元之间的传递,拉马克的进化学说,遗传的分子基础
0,0,下列有关生物体内信息传递的叙述，正确的是（）A下丘脑分泌的促甲状腺激素释放激素，可作用于甲状...,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,从自然菌样筛选较理想生产菌种的一般步骤是：采集菌样→富集培养→纯种分离→性能测定．1.不同微...,1,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,目前，精子载体法逐渐成为具有诱惑力的制备转基因动物方法之一，该方法以精子作为外源基因的载体，...,1,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,某人想乘普通飞机在一年中连续过两次生日，你认为应穿越（）A赤道B两级C本初子午线D国际日期变更线,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,关于太阳辐射对地球影响的叙述，正确的是（）A太阳放出的能量中到达地球的很多，成为地球表面自然...,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
train_size = int(data.shape[0] * 0.9)
train = data[:train_size]
test = data[train_size:]
train.head()

,id,content,高中,生物,历史,地理,政治,生物技术实践,公民道德与伦理常识,经济学常识,...,社会主义是中国人民的历史性选择,人口迁移与人口流动,走进细胞,避孕的原理和方法,血糖平衡的调节,海峡两岸关系的发展,生命活动离不开细胞,兴奋在神经元之间的传递,拉马克的进化学说,遗传的分子基础
0,0,下列有关生物体内信息传递的叙述，正确的是（）A下丘脑分泌的促甲状腺激素释放激素，可作用于甲状...,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,从自然菌样筛选较理想生产菌种的一般步骤是：采集菌样→富集培养→纯种分离→性能测定．1.不同微...,1,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,目前，精子载体法逐渐成为具有诱惑力的制备转基因动物方法之一，该方法以精子作为外源基因的载体，...,1,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,某人想乘普通飞机在一年中连续过两次生日，你认为应穿越（）A赤道B两级C本初子午线D国际日期变更线,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,关于太阳辐射对地球影响的叙述，正确的是（）A太阳放出的能量中到达地球的很多，成为地球表面自然...,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [10]:
ID = 'id'
DATA_COLUMN = 'content'
LABEL_COLUMNS = ['高中', '生物', '历史', '地理', '政治', '生物技术实践', '公民道德与伦理常识', '经济学常识',
              '生活中的法律常识', '现代生物技术专题', '科学社会主义常识',
              '地球与地图', '近代史', '遗传与进化', '生物科学与社会', '分子与细胞', '人口与城市',
              '宇宙中的地球', '生产活动与地域联系', '古代史', '现代史', '稳态与环境',
              '生物性污染', '细胞有丝分裂不同时期的特点', '液泡的结构和功能', '组成细胞的化学元素',
              '兴奋在神经纤维上的传导', '不完全显性', '免疫系统的组成', '生物技术在其他方面的应用', '群落的结构',
              '中央官制——三公九卿制', '核糖体的结构和功能', '人体免疫系统在维持稳态中的作用', '皇帝制度',
              '激素调节', '伴性遗传',
              '地球运动的地理意义', '地球运动的基本形式', '基因工程的原理及技术', '体液免疫的概念和过程',
              '基因的分离规律的实质及应用', '蛋白质的合成', '地球的内部圈层结构及特点', '人口增长与人口问题',
              '劳动就业与守法经营', '器官移植', '垄断组织的出现', '基因工程的概念', '神经调节和体液调节的比较',
              '组成细胞的化合物', '文艺的春天', '生物工程技术', '基因的自由组合规律的实质及应用', '郡县制', '人体水盐平衡调节',
              '内质网的结构和功能', '人体的体温调节', '免疫系统的功能', '与细胞分裂有关的细胞器', '太阳对地球的影响',
              '清末民主革命风潮', '复等位基因', '人工授精、试管婴儿等生殖技术', '“重农抑商”政策', '生态系统的营养结构',
              '减数分裂的概念', '地球的外部圈层结构及特点', '细胞的多样性和统一性', '工业区位因素', '细胞大小与物质运输的关系',
              '夏商两代的政治制度', '农业区位因素', '溶酶体的结构和功能', '内环境的稳态', '胚胎移植',
              '第三产业的兴起和“新经济”的出现', '中心体的结构和功能', '社会主义市场经济的伦理要求', '选官、用官制度的变化',
              '减数分裂与有丝分裂的比较', '遗传的细胞基础', '地球所处的宇宙环境', '培养基与无菌技术', '高尔基体的结构和功能',
              '社会主义是中国人民的历史性选择', '人口迁移与人口流动', '走进细胞', '避孕的原理和方法', '血糖平衡的调节',
              '海峡两岸关系的发展', '生命活动离不开细胞', '兴奋在神经元之间的传递', '拉马克的进化学说', '遗传的分子基础']

In [11]:
class InputExample(object):
    """A single training/test example for simple sequence classification."""

    def __init__(self, guid, text_a, text_b=None, labels=None):
        """Constructs a InputExample.

        Args:
            guid: Unique id for the example.
            text_a: string. The untokenized text of the first sequence. For single
            sequence tasks, only this sequence must be specified.
            text_b: (Optional) string. The untokenized text of the second sequence.
            Only must be specified for sequence pair tasks.
            labels: (Optional) [string]. The label of the example. This should be
            specified for train and dev examples, but not for test examples.
        """
        self.guid = guid
        self.text_a = text_a
        self.text_b = text_b
        self.labels = labels


class InputFeatures(object):
    """A single set of features of data."""

    def __init__(self, input_ids, input_mask, segment_ids, label_ids, is_real_example=True):
        self.input_ids = input_ids
        self.input_mask = input_mask
        self.segment_ids = segment_ids
        self.label_ids = label_ids,
        self.is_real_example=is_real_example

In [12]:
def create_examples(df, labels_available=True):
    """Creates examples for the training and dev sets."""
    examples = []
    for (i, row) in enumerate(df.values):
        guid = row[0]
        text_a = row[1]
        if labels_available:
            labels = row[2:]
        else:
            labels = [0] * len(LABEL_COLUMNS)
        examples.append(
            InputExample(guid=guid, text_a=text_a, labels=labels))
    return examples

In [13]:
TRAIN_VAL_RATIO = 0.9
LEN = train.shape[0]
SIZE_TRAIN = int(TRAIN_VAL_RATIO*LEN)

x_train = train[:SIZE_TRAIN]
x_val = train[SIZE_TRAIN:]

train_examples = create_examples(x_train)

In [14]:
def convert_examples_to_features(examples,  max_seq_length, tokenizer):
    """Loads a data file into a list of `InputBatch`s."""

    features = []
    for (ex_index, example) in enumerate(examples):
        print(example.text_a)
        tokens_a = tokenizer.tokenize(example.text_a)

        tokens_b = None
        if example.text_b:
            tokens_b = tokenizer.tokenize(example.text_b)
            # Modifies `tokens_a` and `tokens_b` in place so that the total
            # length is less than the specified length.
            # Account for [CLS], [SEP], [SEP] with "- 3"
            _truncate_seq_pair(tokens_a, tokens_b, max_seq_length - 3)
        else:
            # Account for [CLS] and [SEP] with "- 2"
            if len(tokens_a) > max_seq_length - 2:
                tokens_a = tokens_a[:(max_seq_length - 2)]

        # The convention in BERT is:
        # (a) For sequence pairs:
        #  tokens:   [CLS] is this jack ##son ##ville ? [SEP] no it is not . [SEP]
        #  type_ids: 0   0  0    0    0     0       0 0    1  1  1  1   1 1
        # (b) For single sequences:
        #  tokens:   [CLS] the dog is hairy . [SEP]
        #  type_ids: 0   0   0   0  0     0 0
        #
        # Where "type_ids" are used to indicate whether this is the first
        # sequence or the second sequence. The embedding vectors for `type=0` and
        # `type=1` were learned during pre-training and are added to the wordpiece
        # embedding vector (and position vector). This is not *strictly* necessary
        # since the [SEP] token unambigiously separates the sequences, but it makes
        # it easier for the model to learn the concept of sequences.
        #
        # For classification tasks, the first vector (corresponding to [CLS]) is
        # used as as the "sentence vector". Note that this only makes sense because
        # the entire model is fine-tuned.
        tokens = ["[CLS]"] + tokens_a + ["[SEP]"]
        segment_ids = [0] * len(tokens)

        if tokens_b:
            tokens += tokens_b + ["[SEP]"]
            segment_ids += [1] * (len(tokens_b) + 1)

        input_ids = tokenizer.convert_tokens_to_ids(tokens)

        # The mask has 1 for real tokens and 0 for padding tokens. Only real
        # tokens are attended to.
        input_mask = [1] * len(input_ids)

        # Zero-pad up to the sequence length.
        padding = [0] * (max_seq_length - len(input_ids))
        input_ids += padding
        input_mask += padding
        segment_ids += padding

        assert len(input_ids) == max_seq_length
        assert len(input_mask) == max_seq_length
        assert len(segment_ids) == max_seq_length
        
        labels_ids = []
        for label in example.labels:
            labels_ids.append(int(label))

        if ex_index < 0:
            logger.info("*** Example ***")
            logger.info("guid: %s" % (example.guid))
            logger.info("tokens: %s" % " ".join(
                    [str(x) for x in tokens]))
            logger.info("input_ids: %s" % " ".join([str(x) for x in input_ids]))
            logger.info("input_mask: %s" % " ".join([str(x) for x in input_mask]))
            logger.info(
                    "segment_ids: %s" % " ".join([str(x) for x in segment_ids]))
            logger.info("label: %s (id = %s)" % (example.labels, labels_ids))

        features.append(
                InputFeatures(input_ids=input_ids,
                              input_mask=input_mask,
                              segment_ids=segment_ids,
                              label_ids=labels_ids))
    return features

In [15]:
# We'll set sequences to be at most 128 tokens long.
MAX_SEQ_LENGTH = 128

# Compute train and warmup steps from batch size
# These hyperparameters are copied from this colab notebook (https://colab.sandbox.google.com/github/tensorflow/tpu/blob/master/tools/colab/bert_finetuning_with_cloud_tpus.ipynb)
BATCH_SIZE = 32
LEARNING_RATE = 5e-5
NUM_TRAIN_EPOCHS = 5.0
# Warmup is a period of time where hte learning rate 
# is small and gradually increases--usually helps training.
WARMUP_PROPORTION = 0.1
# Model configs
SAVE_CHECKPOINTS_STEPS = 1000
SAVE_SUMMARY_STEPS = 500

In [16]:
class PaddingInputExample(object):
    """Fake example so the num input examples is a multiple of the batch size.
    When running eval/predict on the TPU, we need to pad the number of examples
    to be a multiple of the batch size, because the TPU requires a fixed batch
    size. The alternative is to drop the last batch, which is bad because it means
    the entire output data won't be generated.
    We use this class instead of `None` because treating `None` as padding
    battches could cause silent errors.
    """
    
    
def convert_single_example(ex_index, example, max_seq_length,
                           tokenizer):
    """Converts a single `InputExample` into a single `InputFeatures`."""

    if isinstance(example, PaddingInputExample):
        return InputFeatures(
            input_ids=[0] * max_seq_length,
            input_mask=[0] * max_seq_length,
            segment_ids=[0] * max_seq_length,
            label_ids=0,
            is_real_example=False)

    tokens_a = tokenizer.tokenize(example.text_a)
    tokens_b = None
    if example.text_b:
        tokens_b = tokenizer.tokenize(example.text_b)

    if tokens_b:
        # Modifies `tokens_a` and `tokens_b` in place so that the total
        # length is less than the specified length.
        # Account for [CLS], [SEP], [SEP] with "- 3"
        _truncate_seq_pair(tokens_a, tokens_b, max_seq_length - 3)
    else:
        # Account for [CLS] and [SEP] with "- 2"
        if len(tokens_a) > max_seq_length - 2:
            tokens_a = tokens_a[0:(max_seq_length - 2)]

    # The convention in BERT is:
    # (a) For sequence pairs:
    #  tokens:   [CLS] is this jack ##son ##ville ? [SEP] no it is not . [SEP]
    #  type_ids: 0     0  0    0    0     0       0 0     1  1  1  1   1 1
    # (b) For single sequences:
    #  tokens:   [CLS] the dog is hairy . [SEP]
    #  type_ids: 0     0   0   0  0     0 0
    #
    # Where "type_ids" are used to indicate whether this is the first
    # sequence or the second sequence. The embedding vectors for `type=0` and
    # `type=1` were learned during pre-training and are added to the wordpiece
    # embedding vector (and position vector). This is not *strictly* necessary
    # since the [SEP] token unambiguously separates the sequences, but it makes
    # it easier for the model to learn the concept of sequences.
    #
    # For classification tasks, the first vector (corresponding to [CLS]) is
    # used as the "sentence vector". Note that this only makes sense because
    # the entire model is fine-tuned.
    tokens = []
    segment_ids = []
    tokens.append("[CLS]")
    segment_ids.append(0)
    for token in tokens_a:
        tokens.append(token)
        segment_ids.append(0)
    tokens.append("[SEP]")
    segment_ids.append(0)

    if tokens_b:
        for token in tokens_b:
            tokens.append(token)
            segment_ids.append(1)
        tokens.append("[SEP]")
        segment_ids.append(1)

    input_ids = tokenizer.convert_tokens_to_ids(tokens)

    # The mask has 1 for real tokens and 0 for padding tokens. Only real
    # tokens are attended to.
    input_mask = [1] * len(input_ids)

    # Zero-pad up to the sequence length.
    while len(input_ids) < max_seq_length:
        input_ids.append(0)
        input_mask.append(0)
        segment_ids.append(0)

    assert len(input_ids) == max_seq_length
    assert len(input_mask) == max_seq_length
    assert len(segment_ids) == max_seq_length

    labels_ids = []
    for label in example.labels:
        labels_ids.append(int(label))


    feature = InputFeatures(
        input_ids=input_ids,
        input_mask=input_mask,
        segment_ids=segment_ids,
        label_ids=labels_ids,
        is_real_example=True)
    return feature


def file_based_convert_examples_to_features(
        examples, max_seq_length, tokenizer, output_file):
    """Convert a set of `InputExample`s to a TFRecord file."""

    writer = tf.python_io.TFRecordWriter(output_file)

    for (ex_index, example) in enumerate(examples):
        #if ex_index % 10000 == 0:
            #tf.logging.info("Writing example %d of %d" % (ex_index, len(examples)))

        feature = convert_single_example(ex_index, example,
                                         max_seq_length, tokenizer)

        def create_int_feature(values):
            f = tf.train.Feature(int64_list=tf.train.Int64List(value=list(values)))
            return f

        features = collections.OrderedDict()
        features["input_ids"] = create_int_feature(feature.input_ids)
        features["input_mask"] = create_int_feature(feature.input_mask)
        features["segment_ids"] = create_int_feature(feature.segment_ids)
        features["is_real_example"] = create_int_feature(
            [int(feature.is_real_example)])
        if isinstance(feature.label_ids, list):
            label_ids = feature.label_ids
        else:
            label_ids = feature.label_ids[0]
        features["label_ids"] = create_int_feature(label_ids)

        tf_example = tf.train.Example(features=tf.train.Features(feature=features))
        writer.write(tf_example.SerializeToString())
    writer.close()


def file_based_input_fn_builder(input_file, seq_length, is_training,
                                drop_remainder):
    """Creates an `input_fn` closure to be passed to TPUEstimator."""

    # 修改 "label_ids": tf.FixedLenFeature([6], tf.int64)
    name_to_features = {
        "input_ids": tf.FixedLenFeature([seq_length], tf.int64),
        "input_mask": tf.FixedLenFeature([seq_length], tf.int64),
        "segment_ids": tf.FixedLenFeature([seq_length], tf.int64),
        "label_ids": tf.FixedLenFeature([95], tf.int64),
        "is_real_example": tf.FixedLenFeature([], tf.int64),
    }

    def _decode_record(record, name_to_features):
        """Decodes a record to a TensorFlow example."""
        example = tf.parse_single_example(record, name_to_features)

        # tf.Example only supports tf.int64, but the TPU only supports tf.int32.
        # So cast all int64 to int32.
        for name in list(example.keys()):
            t = example[name]
            if t.dtype == tf.int64:
                t = tf.to_int32(t)
            example[name] = t

        return example

    def input_fn(params):
        """The actual input function."""
        batch_size = params["batch_size"]

        # For training, we want a lot of parallel reading and shuffling.
        # For eval, we want no shuffling and parallel reading doesn't matter.
        d = tf.data.TFRecordDataset(input_file)
        if is_training:
            d = d.repeat()
            d = d.shuffle(buffer_size=100)

        d = d.apply(
            tf.contrib.data.map_and_batch(
                lambda record: _decode_record(record, name_to_features),
                batch_size=batch_size,
                drop_remainder=drop_remainder))

        return d

    return input_fn


def _truncate_seq_pair(tokens_a, tokens_b, max_length):
    """Truncates a sequence pair in place to the maximum length."""

    # This is a simple heuristic which will always truncate the longer sequence
    # one token at a time. This makes more sense than truncating an equal percent
    # of tokens from each, since if one sequence is very short then each token
    # that's truncated likely contains more information than a longer sequence.
    while True:
        total_length = len(tokens_a) + len(tokens_b)
        if total_length <= max_length:
            break
        if len(tokens_a) > len(tokens_b):
            tokens_a.pop()
        else:
            tokens_b.pop()

In [17]:
# Compute # train and warmup steps from batch size
num_train_steps = int(len(train_examples) / BATCH_SIZE * NUM_TRAIN_EPOCHS)
num_warmup_steps = int(num_train_steps * WARMUP_PROPORTION)

In [18]:
!mkdir working
train_file = os.path.join('./working', "train.tf_record")
#filename = Path(train_file)
if not os.path.exists(train_file):
    open(train_file, 'w').close()

In [19]:
file_based_convert_examples_to_features(train_examples, MAX_SEQ_LENGTH, tokenizer, train_file)
tf.logging.info("***** Running training *****")
tf.logging.info("  Num examples = %d", len(train_examples))
tf.logging.info("  Batch size = %d", BATCH_SIZE)
tf.logging.info("  Num steps = %d", num_train_steps)

In [20]:
train_input_fn = file_based_input_fn_builder(
    input_file=train_file,
    seq_length=MAX_SEQ_LENGTH,
    is_training=True,
    drop_remainder=True)

In [21]:
def create_model(bert_config, is_training, input_ids, input_mask, segment_ids,
                 labels, num_labels, use_one_hot_embeddings):
    """Creates a classification model."""
    model = modeling.BertModel(
        config=bert_config,
        is_training=is_training,
        input_ids=input_ids,
        input_mask=input_mask,
        token_type_ids=segment_ids,
        use_one_hot_embeddings=use_one_hot_embeddings)

    # In the demo, we are doing a simple classification task on the entire
    # segment.
    #
    # If you want to use the token-level output, use model.get_sequence_output()
    # instead.
    output_layer = model.get_pooled_output()

    hidden_size = output_layer.shape[-1].value

    output_weights = tf.get_variable(
        "output_weights", [num_labels, hidden_size],
        initializer=tf.truncated_normal_initializer(stddev=0.02))

    output_bias = tf.get_variable(
        "output_bias", [num_labels], initializer=tf.zeros_initializer())

    with tf.variable_scope("loss"):
        if is_training:
            # I.e., 0.1 dropout
            output_layer = tf.nn.dropout(output_layer, keep_prob=0.9)

        logits = tf.matmul(output_layer, output_weights, transpose_b=True)
        logits = tf.nn.bias_add(logits, output_bias)
        
        # probabilities = tf.nn.softmax(logits, axis=-1) ### multiclass case
        probabilities = tf.nn.sigmoid(logits)#### multi-label case
        
        labels = tf.cast(labels, tf.float32)
        tf.logging.info("num_labels:{};logits:{};labels:{}".format(num_labels, logits, labels))
        per_example_loss = tf.nn.sigmoid_cross_entropy_with_logits(labels=labels, logits=logits)
        loss = tf.reduce_mean(per_example_loss)

        # probabilities = tf.nn.softmax(logits, axis=-1)
        # log_probs = tf.nn.log_softmax(logits, axis=-1)
        #
        # one_hot_labels = tf.one_hot(labels, depth=num_labels, dtype=tf.float32)
        #
        # per_example_loss = -tf.reduce_sum(one_hot_labels * log_probs, axis=-1)
        # loss = tf.reduce_mean(per_example_loss)

        return (loss, per_example_loss, logits, probabilities)


def model_fn_builder(bert_config, num_labels, init_checkpoint, learning_rate,
                     num_train_steps, num_warmup_steps, use_tpu,
                     use_one_hot_embeddings):
    """Returns `model_fn` closure for TPUEstimator."""

    def model_fn(features, labels, mode, params):  # pylint: disable=unused-argument
        """The `model_fn` for TPUEstimator."""

        #tf.logging.info("*** Features ***")
        #for name in sorted(features.keys()):
        #    tf.logging.info("  name = %s, shape = %s" % (name, features[name].shape))

        input_ids = features["input_ids"]
        input_mask = features["input_mask"]
        segment_ids = features["segment_ids"]
        label_ids = features["label_ids"]
        is_real_example = None
        if "is_real_example" in features:
             is_real_example = tf.cast(features["is_real_example"], dtype=tf.float32)
        else:
             is_real_example = tf.ones(tf.shape(label_ids), dtype=tf.float32)

        is_training = (mode == tf.estimator.ModeKeys.TRAIN)

        (total_loss, per_example_loss, logits, probabilities) = create_model(
            bert_config, is_training, input_ids, input_mask, segment_ids, label_ids,
            num_labels, use_one_hot_embeddings)

        tvars = tf.trainable_variables()
        initialized_variable_names = {}
        scaffold_fn = None
        if init_checkpoint:
            (assignment_map, initialized_variable_names
             ) = modeling.get_assignment_map_from_checkpoint(tvars, init_checkpoint)
            if use_tpu:

                def tpu_scaffold():
                    tf.train.init_from_checkpoint(init_checkpoint, assignment_map)
                    return tf.train.Scaffold()

                scaffold_fn = tpu_scaffold
            else:
                tf.train.init_from_checkpoint(init_checkpoint, assignment_map)

        tf.logging.info("**** Trainable Variables ****")
        for var in tvars:
            init_string = ""
            if var.name in initialized_variable_names:
                init_string = ", *INIT_FROM_CKPT*"
            #tf.logging.info("  name = %s, shape = %s%s", var.name, var.shape,init_string)

        output_spec = None
        if mode == tf.estimator.ModeKeys.TRAIN:

            train_op = optimization.create_optimizer(
                total_loss, learning_rate, num_train_steps, num_warmup_steps, use_tpu)

            output_spec = tf.estimator.EstimatorSpec(
                mode=mode,
                loss=total_loss,
                train_op=train_op,
                scaffold=scaffold_fn)
        elif mode == tf.estimator.ModeKeys.EVAL:

            def metric_fn(per_example_loss, label_ids, probabilities, is_real_example):

                logits_split = tf.split(probabilities, num_labels, axis=-1)
                label_ids_split = tf.split(label_ids, num_labels, axis=-1)
                # metrics change to auc of every class
                eval_dict = {}
                for j, logits in enumerate(logits_split):
                    label_id_ = tf.cast(label_ids_split[j], dtype=tf.int32)
                    current_auc, update_op_auc = tf.metrics.auc(label_id_, logits)
                    eval_dict[str(j)] = (current_auc, update_op_auc)
                eval_dict['eval_loss'] = tf.metrics.mean(values=per_example_loss)
                return eval_dict

                ## original eval metrics
                # predictions = tf.argmax(logits, axis=-1, output_type=tf.int32)
                # accuracy = tf.metrics.accuracy(
                #     labels=label_ids, predictions=predictions, weights=is_real_example)
                # loss = tf.metrics.mean(values=per_example_loss, weights=is_real_example)
                # return {
                #     "eval_accuracy": accuracy,
                #     "eval_loss": loss,
                # }

            eval_metrics = metric_fn(per_example_loss, label_ids, probabilities, is_real_example)
            output_spec = tf.estimator.EstimatorSpec(
                mode=mode,
                loss=total_loss,
                eval_metric_ops=eval_metrics,
                scaffold=scaffold_fn)
        else:
            print("mode:", mode,"probabilities:", probabilities)
            output_spec = tf.estimator.EstimatorSpec(
                mode=mode,
                predictions={"probabilities": probabilities},
                scaffold=scaffold_fn)
        return output_spec

    return model_fn

In [22]:
!mkdir ./working/output
OUTPUT_DIR = "./working/output"
# Specify outpit directory and number of checkpoint steps to save
run_config = tf.estimator.RunConfig(
    model_dir=OUTPUT_DIR,
    save_summary_steps=SAVE_SUMMARY_STEPS,
    keep_checkpoint_max=1,
    save_checkpoints_steps=SAVE_CHECKPOINTS_STEPS)

In [23]:
bert_config = modeling.BertConfig.from_json_file(BERT_CONFIG)
model_fn = model_fn_builder(
  bert_config=bert_config,
  num_labels= len(LABEL_COLUMNS),
  init_checkpoint=BERT_INIT_CHKPNT,
  learning_rate=LEARNING_RATE,
  num_train_steps=num_train_steps,
  num_warmup_steps=num_warmup_steps,
  use_tpu=False,
  use_one_hot_embeddings=False)

estimator = tf.estimator.Estimator(
  model_fn=model_fn,
  config=run_config,
  params={"batch_size": BATCH_SIZE})

In [24]:
print(f'Beginning Training!')
current_time = datetime.now()
estimator.train(input_fn=train_input_fn, max_steps=num_train_steps)
print("Training took time ", datetime.now() - current_time)

Beginning Training!
Training took time  0:23:56.572701


In [25]:
eval_file = os.path.join('./working', "eval.tf_record")
#filename = Path(train_file)
if not os.path.exists(eval_file):
    open(eval_file, 'w').close()

eval_examples = create_examples(x_val)
file_based_convert_examples_to_features(
    eval_examples, MAX_SEQ_LENGTH, tokenizer, eval_file)

In [26]:
# This tells the estimator to run through the entire set.
eval_steps = None

eval_drop_remainder = False
eval_input_fn = file_based_input_fn_builder(
    input_file=eval_file,
    seq_length=MAX_SEQ_LENGTH,
    is_training=False,
    drop_remainder=False)

result = estimator.evaluate(input_fn=eval_input_fn, steps=eval_steps)

In [27]:

output_eval_file = os.path.join("./working", "eval_results.txt")
with tf.gfile.GFile(output_eval_file, "w") as writer:
    tf.logging.info("***** Eval results *****")
    for key in sorted(result.keys()):
        tf.logging.info("  %s = %s", key, str(result[key]))
        writer.write("%s = %s\n" % (key, str(result[key])))

In [28]:
# x_test = test[:10000] #testing a small sample
x_test = test
x_test = x_test.reset_index(drop=True)
predict_examples = create_examples(x_test,False)

In [29]:
test_features = convert_examples_to_features(predict_examples, MAX_SEQ_LENGTH, tokenizer)

清代定制，官员任命由军机处等机构简选，最后由皇帝朱笔圈定简放。但自太平天国运动以来，不仅司道以下官员的任命，皆由督抚奏定，而且邻省督抚的任命，有时也要征求有力督抚的意见。这表明（）A.皇帝的权力受到了限制B.统治危机冲击中央集权C.清政府已无力控制地方D.地方政府掌握了行政权
我国北方住宅区的楼房间距理论上应比南方宽，理由是（）A北方正午太阳高度比南方小B北方冬季白昼时间比南方短C南方气候比北方潮湿D北方平原比南方开阔
植物体细胞杂交能解决不同植物之间由于生殖隔离而不能进行有性杂交的问题．植物体细胞杂交过程不涉及（）A使用限制性核酸内切酶B使用纤维素酶和果胶酶C诱导两种不同的细胞融合D选择利于杂交细胞分裂和分化的培养基
1215年6月15日，安茹王朝屡战屡败的国王约翰与25位贵族代表举行谈判，签署了著名的《自由大宪章》，从此英国确立“王在议会”和“王在法下”的原则。1603年伊丽莎白一世去世，她远在外国的侄孙詹姆士一世继位，詹姆士对英格兰一无所知，坚持“君权神授”，王权逐步凌驾于议会之上。直到光荣革命的胜利，议会才“收复”失去的权力。这表明，光荣革命（）A.对英格兰的政治传统有所捍卫B.结束了英国君主制的历史进程C.标志着英国君主立宪制的确立D.使英国国王处于“统而不治”地位
下列关于基因工程的叙述，正确的是（）A基因工程经常以抗菌素抗性基因为目的基因B细菌质粒是基因工程常用的运载体C通常用一种限制性内切酶处理含目的基因的DNA，用另一种处理运载体DNAD为育成抗除草剂的作物新品种，导入抗除草剂基因时只能以受精卵为受体
地质学家常利用地震波来寻找矿产资源，据下图A处地震波波速变化信息看，在A处最有可能储藏的矿产为（）A铁矿B有色金属C石油D煤
在如图所示的胚芽鞘中，生长素的运输方向一般不会发生的是（）A.a→bB.b→dC.c→dD.a→c
如图是闰绍细胞（一种抑制性中间神经元）参与调节的过程．下列相关叙述中，不正确的是（）A运动神经元1兴奋时，通过闰绍细胞抑制自身的活动B闰绍细胞兴奋时会抑制运动神经元2的兴奋性C图示神经元之间的环状联系，是一种负反馈调节D闰绍细胞通过递质使运动神经元1膜电位变为内正外负
太阳活动最激烈的显示是（）A太阳黑子B耀斑C太阳风D日珥爆发
从2010年的经济政策可以看出我国政府更多地运用税收、利率等经济杠杆调节经济。经济杠杆能够起到调节作

In [30]:
def input_fn_builder(features, seq_length, is_training, drop_remainder):
  """Creates an `input_fn` closure to be passed to TPUEstimator."""

  all_input_ids = []
  all_input_mask = []
  all_segment_ids = []
  all_label_ids = []

  for feature in features:
    all_input_ids.append(feature.input_ids)
    all_input_mask.append(feature.input_mask)
    all_segment_ids.append(feature.segment_ids)
    all_label_ids.append(feature.label_ids)

  def input_fn(params):
    """The actual input function."""
    batch_size = params["batch_size"]

    num_examples = len(features)

    # This is for demo purposes and does NOT scale to large data sets. We do
    # not use Dataset.from_generator() because that uses tf.py_func which is
    # not TPU compatible. The right way to load data is with TFRecordReader.
    d = tf.data.Dataset.from_tensor_slices({
        "input_ids":
            tf.constant(
                all_input_ids, shape=[num_examples, seq_length],
                dtype=tf.int32),
        "input_mask":
            tf.constant(
                all_input_mask,
                shape=[num_examples, seq_length],
                dtype=tf.int32),
        "segment_ids":
            tf.constant(
                all_segment_ids,
                shape=[num_examples, seq_length],
                dtype=tf.int32),
        "label_ids":
            tf.constant(all_label_ids, shape=[num_examples, len(LABEL_COLUMNS)], dtype=tf.int32),
    })

    if is_training:
      d = d.repeat()
      d = d.shuffle(buffer_size=100)

    d = d.batch(batch_size=batch_size, drop_remainder=drop_remainder)
    return d

  return input_fn

In [31]:
print('Beginning Predictions!')
current_time = datetime.now()

predict_input_fn = input_fn_builder(features=test_features, seq_length=MAX_SEQ_LENGTH, is_training=False, drop_remainder=False)
predictions = estimator.predict(predict_input_fn)
print("Prediction took time ", datetime.now() - current_time)

Beginning Predictions!
Prediction took time  0:00:00.001994


In [32]:

def create_output(predictions):
    probabilities = []
    for (i, prediction) in enumerate(predictions):
        preds = prediction["probabilities"]
        probabilities.append(preds)
    dff = pd.DataFrame(probabilities)
    dff.columns = LABEL_COLUMNS
    return dff

In [33]:
output_df = create_output(predictions)
merged_df =  pd.concat([x_test, output_df], axis=1)

mode: infer probabilities: Tensor("loss/Sigmoid:0", shape=(?, 95), dtype=float32)


In [34]:
submission = merged_df.drop(['content'], axis=1)
submission.to_csv("sample_submission0.csv", index=False)

In [35]:
submission.head()

,id,高中,生物,历史,地理,政治,生物技术实践,公民道德与伦理常识,经济学常识,生活中的法律常识,...,社会主义是中国人民的历史性选择,人口迁移与人口流动,走进细胞,避孕的原理和方法,血糖平衡的调节,海峡两岸关系的发展,生命活动离不开细胞,兴奋在神经元之间的传递,拉马克的进化学说,遗传的分子基础
0,20317,1,0,1,0,0,0,0,0,0,...,0.009298,0.012057,0.004763,0.005407,0.007456,0.031126,0.005834,0.008498,0.005759,0.004627
1,20318,1,0,0,1,0,0,0,0,0,...,0.003560,0.004040,0.002456,0.003694,0.001817,0.002245,0.002606,0.002070,0.001724,0.002571
2,20319,1,1,0,0,0,1,0,0,0,...,0.002591,0.003160,0.005245,0.008292,0.005276,0.002275,0.009401,0.002018,0.013638,0.004654
3,20320,1,0,1,0,0,0,0,0,0,...,0.009748,0.007083,0.005473,0.009764,0.002759,0.752197,0.005538,0.002662,0.006630,0.002930
4,20321,1,1,0,0,0,0,0,0,0,...,0.006314,0.003494,0.006047,0.011357,0.004862,0.004086,0.011884,0.004414,0.017014,0.009686


In [36]:
# 评价指标
def f1_np(y_true, y_pred):
    """F1 metric.

    Computes the micro_f1 and macro_f1, metrics for multi-label classification of
    how many relevant items are selected.
    """
    true_positives = np.sum(np.round(np.clip(y_true * y_pred, 0, 1)), axis=0)
    predicted_positives = np.sum(np.round(np.clip(y_pred, 0, 1)), axis=0)
    possible_positives = np.sum(np.round(np.clip(y_true, 0, 1)), axis=0)

    """Macro_F1 metric.
    """
    precision = true_positives / (predicted_positives + 1e-8)
    recall = true_positives / (possible_positives + 1e-8)
    macro_f1 = np.mean(2 * precision * recall / (precision + recall + 1e-8))

    """Micro_F1 metric.
    """
    precision = np.sum(true_positives) / np.sum(predicted_positives)
    recall = np.sum(true_positives) / np.sum(possible_positives)
    micro_f1 = 2 * precision * recall / (precision + recall + 1e-8)
    print("micro_f1:",micro_f1)
    print("macro_f1:",macro_f1)
    return micro_f1, macro_f1



In [37]:
output_df[output_df>=0.2] = 1
output_df[output_df<0.2] = 0
y_pred = output_df

y_true = x_test.iloc[:,2:]
f1_np(y_true, y_pred)

micro_f1: 0.9060472852399325
macro_f1: 0.8213468989586507


(0.9060472852399325, 0.8213468989586507)

In [38]:
# test.head()